In [1]:
from transformers import pipeline
pipe = pipeline("feature-extraction", model="BAAI/bge-small-zh")

/Users/yukun/Desktop/mda/mdatopic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [ ]:
# ========== 1. 读取所有 txt 文件 ==========
root_path = "/Users/yukun/Desktop/mda/MDA训练集"
file_list = sorted([f for f in os.listdir(root_path) if f.endswith('.txt')])
documents = []

for fname in file_list:
    with open(os.path.join(root_path, fname), 'r', encoding='utf-8') as f:
        text = f.read()
        documents.append((fname, text))

In [ ]:
# ========== 2. 切句 ==========
def split_sentences(text):
    return [s.strip() for s in re.split(r"[。！？\n]", text) if len(s.strip()) > 10]

all_sentences = []
doc_indices = []

for idx, (fname, text) in enumerate(documents):
    sents = split_sentences(text)
    all_sentences.extend(sents)
    doc_indices.extend([idx] * len(sents))  # 记录句子属于哪个文档

In [ ]:
# ========== 3. 加载 HuggingFace pipeline 作为 embedding ==========
from transformers import pipeline
from sklearn.base import BaseEstimator
import numpy as np

class HFTransformerEmbedding(BaseEstimator):
    def __init__(self, pipe):
        self.pipe = pipe

    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        embeddings = []
        for doc in tqdm(documents, desc="Embedding Sentences"):
            features = self.pipe(doc, truncation=True, padding=True)
            if isinstance(features, list):
                features = np.mean(features[0], axis=0)
            embeddings.append(features)
        return np.array(embeddings)

pipe = pipeline("feature-extraction", model="BAAI/bge-small-zh")
embedding_model = HFTransformerEmbedding(pipe)

In [ ]:
# ========== 4. 建立 BERTopic 模型 ==========
topic_model = BERTopic(embedding_model=embedding_model, language="chinese", verbose=True)
topics, probs = topic_model.fit_transform(all_sentences)

In [ ]:

# ========== 5. 每篇文档提取主题相关句子 ==========
num_topics = min(10, len(set(topics)) - 1)  # -1是无关类
docs = [[] for _ in range(len(documents))]  # 每个文档一组句子列表

for tid in range(num_topics):
    for sent, topic_id, doc_id in zip(all_sentences, topics, doc_indices):
        if topic_id == tid:
            docs[doc_id].append((tid, sent))

In [ ]:

# ========== 6. 构造结构化输出 DataFrame ==========
result = pd.DataFrame()
result["文件名"] = [x[0] for x in documents]
result["原文摘要"] = [x[1][:300] for x in documents]  # 原文截断显示

# 每个主题输出一列
for tid in range(num_topics):
    topic_words = topic_model.get_topic(tid)
    topic_name = f"主题{tid}：" + "_".join([w for w, _ in topic_words[:3]])
    result[topic_name] = ["".join([s for t, s in doc if t == tid]) for doc in docs]

In [ ]:
# ========== 7. 保存 ==========
output_path = "/Users/yukun/Desktop/mda/客观分析预处理/客观分析/结构化主题输出.csv"
result.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 分析完成，结果保存至：{output_path}")# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model="BAAI/bge-small-zh")

In [3]:
import os
import re
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ========== 1. 读取所有 txt 文件 ==========
root_path = "/Users/yukun/Desktop/mda/MDA训练集"
file_list = sorted([f for f in os.listdir(root_path) if f.endswith('.txt')])
documents = []

for fname in file_list:
    with open(os.path.join(root_path, fname), 'r', encoding='utf-8') as f:
        text = f.read()
        documents.append((fname, text))

# ========== 2. 切句 ==========
def split_sentences(text):
    return [s.strip() for s in re.split(r"[。！？\n]", text) if len(s.strip()) > 10]

all_sentences = []
doc_indices = []

for idx, (fname, text) in enumerate(documents):
    sents = split_sentences(text)
    all_sentences.extend(sents)
    doc_indices.extend([idx] * len(sents))  # 记录句子属于哪个文档

# ========== 3. 加载 HuggingFace pipeline 作为 embedding ==========
from transformers import pipeline
from sklearn.base import BaseEstimator
import numpy as np

class HFTransformerEmbedding(BaseEstimator):
    def __init__(self, pipe):
        self.pipe = pipe

    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        embeddings = []
        for doc in tqdm(documents, desc="Embedding Sentences"):
            features = self.pipe(doc, truncation=True, padding=True)
            if isinstance(features, list):
                features = np.mean(features[0], axis=0)
            embeddings.append(features)
        return np.array(embeddings)

pipe = pipeline("feature-extraction", model="BAAI/bge-small-zh")
embedding_model = HFTransformerEmbedding(pipe)

# ========== 4. 建立 BERTopic 模型 ==========
topic_model = BERTopic(embedding_model=embedding_model, language="chinese", verbose=True)
topics, probs = topic_model.fit_transform(all_sentences)

# ========== 5. 每篇文档提取主题相关句子 ==========
num_topics = min(10, len(set(topics)) - 1)  # -1是无关类
docs = [[] for _ in range(len(documents))]  # 每个文档一组句子列表

for tid in range(num_topics):
    for sent, topic_id, doc_id in zip(all_sentences, topics, doc_indices):
        if topic_id == tid:
            docs[doc_id].append((tid, sent))

# ========== 6. 构造结构化输出 DataFrame ==========
result = pd.DataFrame()
result["文件名"] = [x[0] for x in documents]
result["原文摘要"] = [x[1][:300] for x in documents]  # 原文截断显示

# 每个主题输出一列
for tid in range(num_topics):
    topic_words = topic_model.get_topic(tid)
    topic_name = f"主题{tid}：" + "_".join([w for w, _ in topic_words[:3]])
    result[topic_name] = ["".join([s for t, s in doc if t == tid]) for doc in docs]

# ========== 7. 保存 ==========
output_path = "/Users/yukun/Desktop/mda/客观分析预处理/客观分析/结构化主题输出.csv"
result.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 分析完成，结果保存至：{output_path}")# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model="BAAI/bge-small-zh")

Device set to use mps:0
2025-06-07 20:46:01,621 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 145/145 [00:08<00:00, 17.30it/s]
2025-06-07 20:46:14,858 - BERTopic - Embedding - Completed ✓
2025-06-07 20:46:14,860 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-07 20:46:16,996 - BERTopic - Dimensionality - Completed ✓
2025-06-07 20:46:16,998 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-07 20:46:17,097 - BERTopic - Cluster - Completed ✓
2025-06-07 20:46:17,105 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-07 20:46:17,233 - BERTopic - Representation - Completed ✓


✅ 分析完成，结果保存至：/Users/yukun/Desktop/mda/客观分析预处理/客观分析/结构化主题输出.csv


Device set to use mps:0
